In [ ]:
#r "BoSSSpad.dll"
// #r ".\binaries\XNSEC.dll"
// #r "C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
//#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
Init();

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Unable to open database: 
primary path: d:\bosss_db_Natconv
No alternative paths specified.
IOException caught while opening database #0 at 'd:\bosss_db_Natconv': Unable to open database - all given paths either don't exist or are ruled out by the machine filter on this machine with name pcmit30 .
Unable to open database: 
primary path: \\hpccluster\hpccluster-scratch\gutierrez\NatConvStudyTEST
No alternative paths specified.
IOException caught while opening database #1 at '\\hpccluster\hpccluster-scratch\gutierrez\NatConvStudyTEST': Unable to open database - all given paths either don't exist or are ruled out by the machine filter on this machine with name pcmit30 .
Unable to open database: 
primary path: \\hpccluster\hpccluster-scratch\gutierrez\bosss_db_Natconv
No alternative paths specified.
IOException caught while opening database #2 at '\\hpccluster\hpccluste

Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 170
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#73.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
using BoSSS.Application.XNSEC;

In [ ]:
BoSSSshell.WorkflowMgm.Init("HeatedCavity_ConvergenceStudy");

Project name is set to 'HeatedCavity_ConvergenceStudy'.
NotSupportedException caught during creation/opening of default database: `AllowedDatabasesPaths` not specified, unable to create Database (should be specified in ~/.BoSSS/etc/BatchProcessorConfig.json)..


In [ ]:
var myBatch = BoSSSshell.GetDefaultQueue();
// var myDb = myBatch.CreateOrOpenCompatibleDatabase("HeatedCavity_ConvergenceStudy");
var myDb = OpenOrCreateDatabase(@"C:\Databases\HeatedCavity_ConvergenceStudy");

Opening existing database 'C:\Databases\HeatedCavity_ConvergenceStudy'.


In [ ]:
// Find number of cells used in the finest mesh
int NCellsFinestMesh = 0;
var allsess = myDb.Sessions;
foreach(var sess in allsess){
    int c = Convert.ToInt32(sess.KeysAndQueries["Grid:NoOfCells"]);
    if( c > NCellsFinestMesh)
    NCellsFinestMesh = c;
}

In [ ]:
NCellsFinestMesh

4096

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
BoSSSshell.WorkflowMgm.hConvergence.Update();

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable; // The session table has to be created again in order to see the results from hconvergence

In [ ]:
int counter = 0;
string[] varNames = new string[]{"VelocityX", "VelocityY", "Pressure","Temperature"};

IEnumerable<KeyValuePair<string, double>>[] RegressionS = new IEnumerable<KeyValuePair<string, double>>[varNames.Length];

    Plot2Ddata[,] PlotTable = new Plot2Ddata[2, 2];
    int cnt2 = -1;
    for (int iCol = 0; iCol < 2; iCol++) {
        for (int iRow = 0; iRow < 2; iRow++) {
            var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
            var allPointTypes = Enum.GetValues(typeof(PointTypes)).Cast<PointTypes>().ToArray();
            PointTypes[] myPointTypes = new PointTypes[]{ PointTypes.Diamond, PointTypes.Box,  PointTypes.LowerTriangle, };
            cnt2++;
            // BoSSS data
            var fmt = new PlotFormat();
            fmt.Style = Styles.Lines;
            fmt.LineColor = allColors[cnt2 % allColors.Length];
            fmt.PointType =  allPointTypes[cnt2 % allPointTypes.Length];
            fmt.DashType = DashTypes.Solid;            
            
            string dgDegStr = (varNames[cnt2] == "VelocityX"  || varNames[cnt2] == "VelocityY" ) ? "DGdegree:Velocity*":"DGdegree:"+varNames[cnt2];
            var Tab = BoSSSshell.WorkflowMgm.SessionTable;
            var Tab2 = Tab.ExtractColumns( dgDegStr,"Grid:hMin","Grid:NoOfCells" ,"L2Error_"+  varNames[cnt2]);
            //Extract last point, the one against we are comparing our solutions with
            Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> Convert.ToInt32(RowEntries["Grid:NoOfCells"]) != NCellsFinestMesh );
            var plot = Tab2.ToPlot("Grid:hMin","L2Error_"+  varNames[cnt2],  dgDegStr);

            int cc = 0;
            foreach(var dataGroup in plot.dataGroups){
                dataGroup.Format.PointType = myPointTypes[cc];
                cc++;
            }            
            plot.LogX = true;
            plot.LogY = true;

            plot.Title = varNames[cnt2];
            plot.TitleFont = 20;
            plot.Xlabel = "h";  
            plot.Ylabel = "L_2 Error";  
            PlotTable[iCol, iRow] = plot;

            RegressionS[cnt2] = plot.Regression();
        }
    }



In [ ]:
var gp = PlotTable.ToGnuplot();
gp.PlotSVG(xRes:1000,yRes:1000)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 VelocityX 
 
 
 DGdegree:Velocity*3 
 
 
 DGdegree:Velocity*3 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*2 
 
 
 DGdegree:Velocity*2 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*1 
 
 
 DGdegree:Velocity*1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 VelocityY 
 
 
 DGdegree:Velocity*3 
 
 
 DGdegree:Velocity*3 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*2 
 
 
 DGdegree:Velocity*2 
 
 
 
 
 
 
 
 
 DGdegree:Velocity*1 
 
 
 DGdegree:Velocity*1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 Pressure 
 
 
 DGdegree:Pressure2 
 
 
 DGdegree:Pressure2 
 
 
 
 
 
 
 
 
 DGdegree:Pressure1 
 
 
 DGdegree:Pressure1 
 
 
 
 
 
 
 
 
 DGdegree:Pressure0 
 
 
 DGdegree:Pressure0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -5 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -3 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 L 2 Error 
 
 
 
 
 h 
 
 
 
 
 Temperature 
 
 
 DGdegree:Temperature3 
 
 
 DGdegree:Temperature3 
 
 
 
 
 
 
 
 
 DGdegree:Temperature2 
 
 
 DGdegree:Temperature2 
 
 
 
 
 
 
 
 
 DGdegree:Temperature1 
 
 
 DGdegree:Temperature1

In [ ]:
// 
int cnt = 0;
foreach(var reg in RegressionS){
    Console.WriteLine(varNames[cnt]);
    var regB = reg.ToArray();
    for(int j = 0; j < regB.Length; j++){       
        int dgDeg = ( regB[j].Key).Last() - '0';
        Console.Write("DG degree: "+ dgDeg+ " \t");
        double slope = regB[j].Value;
        Console.Write("Slope: " +slope + "\t");

        if((slope - (dgDeg + 1) ) >= -0.2)
        Console.WriteLine("Expected slope archieved");
        else
        Console.WriteLine("The slope value is too low");
    }
    cnt++;
}


VelocityX
DG degree: 3 	Slope: 3.659456757406345	The slope value is too low
DG degree: 2 	Slope: 3.616147642370523	Expected slope archieved
DG degree: 1 	Slope: 1.9355884228150229	Expected slope archieved
VelocityY
DG degree: 3 	Slope: 3.8466780298791985	Expected slope archieved
DG degree: 2 	Slope: 3.497448478980067	Expected slope archieved
DG degree: 1 	Slope: 1.9267843475721205	Expected slope archieved
Pressure
DG degree: 2 	Slope: -0.379666437916206	The slope value is too low
DG degree: 1 	Slope: 2.203109085407051	Expected slope archieved
DG degree: 0 	Slope: -0.2421016829868629	The slope value is too low
Temperature
DG degree: 3 	Slope: 4.8020048460418	Expected slope archieved
DG degree: 2 	Slope: 3.9304986124824492	Expected slope archieved
DG degree: 1 	Slope: 1.883114377668935	Expected slope archieved
